In [2]:
import nbimporter

# Import main functionality
#from GPDM_direct_fixedpoints import *
from Experiment_2d_MachensBrody import *

plotly.offline.init_notebook_mode(connected=True)

# Saving outputs and timing
import pickle, datetime, time

from joblib import Parallel, delayed
import multiprocessing
import itertools

In [3]:
def save_csv(fname, arr):
    f_handle = open("../../Text/ICML2018/FIGURES/CSV/"+fname, 'wb')
    np.savetxt(f_handle, arr, delimiter=",")
    f_handle.close()

# Load the data and fits

In [14]:
data_loading = scipy.io.loadmat('../Matlab/Machens_Brody/MachensBrodySim_20180207T130526_loading.mat', squeeze_me=True)
data_decision = scipy.io.loadmat('../Matlab/Machens_Brody/MachensBrodySim_20180207T130526_decision.mat', squeeze_me=True)
excitation_values = [0] + range(25, 76, 5) + [100]
data_excitation = [scipy.io.loadmat('../Matlab/Machens_Brody/MachensBrodySim_20180207T130526_mfsim_noise_001_exc_%0.3d.mat' % (excval), squeeze_me=True)
                     for excval in excitation_values]

In [15]:
start_time = '20180207T133635'
nbatches = 1
excitation_values_true = 1.5 + 0.01*np.array(excitation_values)
excitation_set = excitation_values

# Load the saved files
def load_GP_fit(dataset_ind,batchnum, start_time):
    save_fname_params = "_excitation_%0.3d_batch_%0.2d" % (excitation_values[dataset_ind], batchnum)
    save_fname = "Experiment_machens_results/machens_excitation_" + start_time + save_fname_params + ".pkl"
   
    try:
        # Load and store results
        results_file = pickle.load(open(save_fname, 'r'))
    except:
        results_file = []
    
    return results_file

GP_fit_saves = Parallel(n_jobs=1, verbose=5)(
    delayed(load_GP_fit)(dataset_ind,batchnum, start_time) for batchnum, dataset_ind  in itertools.product(
        range(nbatches), range(len(excitation_set)))
)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  13 out of  13 | elapsed:    0.5s finished


# Define plotting functions

In [62]:
import plotly.figure_factory

def machens_callback_plot_external(pvec_partial, 
                                  opt_params, init_paramvec, transforms, dict_ind, dict_shape, nullclines=None, 
                                   fp_size=None, fp_color=None
                                 ):
    
    paramvec = replace_params(pvec_partial, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)
       
    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        paramdict.values()[:12]
    
    if np.any(np.isnan(lengthscales)):
        set_trace()
    
    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']

    # Plot transition function
    xtmp, ytmp = np.meshgrid(np.linspace(np.min(z[0,:]),np.max(z[0,:]),16),
                             np.linspace(np.min(z[1,:]),np.max(z[1,:]),16))
    xstar = np.concatenate([xtmp.flatten()[:,None], ytmp.flatten()[:,None]], axis=1).T

    L, targets, params = fp_get_static_K(eta=kernel_variance, lengthscales=lengthscales, z=z, u=u, s=s, J=J, 
                                         sig_eps=Sigma_eps, sig_u=Sigma_u, sig_s=Sigma_s, sig_J = Sigma_J)
    mu_star, sig_star, K_pred = fp_predict(xstar, L, targets, params)

    #print(time_full_iter(pvec, y, D, Nz, Ns)[0])
    
    quiver_fig = plotly.figure_factory.create_quiver(np.squeeze(xstar[0,:]), 
                                                     np.squeeze(xstar[1,:]), 
                                                     np.squeeze(mu_star[0,:]-xstar[0,:]), 
                                                     np.squeeze(mu_star[1,:]-xstar[1,:]),
                                                    scale=.25,
                                                    arrow_scale=.4,)

#     quiver_fig = plotly.figure_factory.create_quiver(np.squeeze(y_t1_rsh[0,:]), 
#                                                  np.squeeze(y_t1_rsh[1,:]), 
#                                                  np.squeeze(y_t_rsh[0,:]-y_t1_rsh[0,:]), 
#                                                  np.squeeze(y_t_rsh[1,:]-y_t1_rsh[1,:]),
#                                                 scale=.25,
#                                                 arrow_scale=.4,)
    
    # Add points to figure
    
    # Map Sigma_s values to the range 12-22
    if fp_size is None:
        if Sigma_s.size>1:
            FP_SIZE = -np.squeeze(np.log(Sigma_s))
            FP_SIZE = FP_SIZE - np.min(FP_SIZE)
            FP_SIZE = (FP_SIZE/np.max(FP_SIZE))*10. + 12.
        else:
            FP_SIZE = 12.
    else:
        FP_SIZE=fp_size
        
    if fp_color is None:
        fp_color = 'green'
    
#     # Inducing point locations
#     quiver_fig['data'].append(
#         plt_type.Scatter(x=np.atleast_1d(np.squeeze(z[0,:])), y=np.atleast_1d(np.squeeze(z[1,:])), 
#                          mode='markers', name="Inducing loc", marker=dict(size=10)))
    
    # Estimated fixed points
    quiver_fig['data'].append(
        plt_type.Scatter(x=np.atleast_1d(np.squeeze(s[0,:])), y=np.atleast_1d(np.squeeze(s[1,:])), 
                         mode='markers', name="Fixed loc", marker=dict(size=FP_SIZE, color=fp_color)))
        
    # Add nullclines    
    if nullclines is not None:
        quiver_fig['data'].append(plt_type.Scatter(x=nullclines['nullcline1'][0,:], y=nullclines['nullcline1'][1,:]))
        quiver_fig['data'].append(plt_type.Scatter(x=nullclines['nullcline2'][0,:], y=nullclines['nullcline2'][1,:]))
        
    
    # Add layout
    quiver_fig['layout'] = plt_type.Layout(xaxis={'range': [-1.,8.]}, yaxis={'range': [-1.,8.]})
    
    
    plt(quiver_fig)
    
    return [xstar.T, (mu_star-xstar).T, #Quiver plot
            nullclines['nullcline1'].T, #Nullcline 1
            nullclines['nullcline2'].T, #Nullcline 2
            s.T, FP_SIZE[:,None], fp_color] # Fixed points

In [76]:
def plotSingle(curFit, curData=None):
    [y_train, x_train,
     all_results, simParams,
     init_paramvec, dict_ind, dict_shape, opt_params, 
     bnds, transforms] = curFit
    
    paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        paramdict.values()[:12]

    if np.any(np.isnan(lengthscales)):
        set_trace()

    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']
        
        
    fp_p_vals = fp_dynamic_threshold(curFit)
    fp_size = fp_p_vals - np.min(fp_p_vals)
    fp_size = fp_size/np.max(fp_size)
    fp_size = np.squeeze(fp_size*12+8)

    cur_eigs=np.zeros((paramdict['J'].shape[0],2))
    
    color_arr = np.array(['red', 'blue', 'black'])
    color_choice = np.squeeze(np.int32(np.empty(paramdict['Sigma_s'].shape)))
    for i in range(paramdict['J'].shape[0]):
        cur_eigs[i,:] = np.sort(np.linalg.eig(paramdict['J'][i,:,:])[0])
        color_choice[i] = np.int32(cur_eigs[i,1]>1)
    #color_choice[np.squeeze(paramdict['Sigma_s'] > fp_dynamic_threshold(curFit))] = 2
    
        
    quiver_fig = machens_callback_plot_external(all_results[-1].x, 
                          opt_params, init_paramvec, transforms, dict_ind, dict_shape,
                                   nullclines=curData, fp_size=fp_size, fp_color = color_arr[color_choice]
                         )
    
    return [quiver_fig, cur_eigs]

### Figure 3A - Show GP fit with training data

In [86]:
for i in [0,6,12]:
    [a, cur_eigs] = plotSingle(GP_fit_saves[i], data_excitation[i])

    # We need the traces for training data, GP variances and fixed points

    # Line1 - Quiver plot (simply coordinates, do styling in latex)
    save_csv(("figure3_panel1_line1_ind%0.2d" % (i)) + ".csv", np.concatenate([a[0], a[1]],axis=1))

    # Line2 - Nullcline1
    save_csv(("figure3_panel1_line2_ind%0.2d" % (i)) + ".csv", a[2])

    # Line3 - Nullcline2
    save_csv(("figure3_panel1_line3_ind%0.2d" % (i)) + ".csv", a[3])

    # Line4 - Fixed point coordinates, sizes as 3rd column and (color as 4th column) # Python plot marker size: a[-2].marker.size[:,None]
    save_csv(("figure3_panel1_line4_ind%0.2d" % (i)) + ".csv", np.concatenate([a[4], a[5], cur_eigs],axis=1))

# Figure 3B - Eigenvalues of middle point

In [88]:
excitation_values_true = 1.5 + 0.01*np.array(excitation_values)

array([ 1.5 ,  1.75,  1.8 ,  1.85,  1.9 ,  1.95,  2.  ,  2.05,  2.1 ,
        2.15,  2.2 ,  2.25,  2.5 ])

In [121]:
all_eigvals = np.zeros((0,2))
mean_evecs = np.zeros((2,2))

for i in range(len(GP_fit_saves)):
    
    [y_train, x_train,
     all_results, simParams,
     init_paramvec, dict_ind, dict_shape, opt_params, 
     bnds, transforms] = GP_fit_saves[i]

    paramvec = replace_params(all_results[-1].x, opt_params, init_paramvec)
    paramdict = vec_to_params(paramvec, dict_ind, dict_shape, transforms)

    # Unpack the usual parameters
    (Sigma_eps, mu_0_0, Sigma_0_0, C, Sigma_nu, z, u, Sigma_u, lengthscales, kernel_variance, s, J)  = \
        paramdict.values()[:12]

    if np.any(np.isnan(lengthscales)):
        set_trace()

    # Deal with the extra possible parameters
    Sigma_s = None; Sigma_J=None;
    if 'Sigma_s' in paramdict.keys():
        Sigma_s = paramdict['Sigma_s']
    if 'Sigma_J' in paramdict.keys():
        Sigma_J = paramdict['Sigma_J']

    # Get the middle fixed point in every case
    mid_ind = np.argsort((s[0,:]-s[1,:])**2)[0]
    
    #print s[:,mid_ind], mid_ind
    
    [evs, evecs] = np.linalg.eig(paramdict['J'][mid_ind,:,:])
    ev_inds = np.argsort(evs)
    
    all_eigvals = np.concatenate([all_eigvals, evs[ev_inds,None].T], axis=0)
    mean_evecs = mean_evecs + evecs[:,ev_inds]
    
mean_evecs = mean_evecs / (1.0*len(GP_fit_saves))

# Line1 and 2 - Eigenvalues of middle point vs excitation (simply coordinates, do styling in latex)
save_csv("figure3_panel2_line1" + ".csv", np.concatenate([excitation_values_true[:,None], all_eigvals],axis=1))

# Line 3 - Mean eigenvector directions
save_csv("figure3_panel2_line2" + ".csv", np.concatenate([np.zeros((2,2)), mean_evecs.T],axis=1))


In [123]:
mean_evecs

array([[ 0.26472482,  0.70432248],
       [ 0.27905318, -0.70710037]])

In [ ]:
# Line5 - Fixed point coordinates, sizes as 3rd column and (color as 4th column) # Python plot marker size: a[-2].marker.size[:,None]
save_csv("figure2_panel3_line1.csv", np.concatenate([all_r_vals, all_x_vals, all_sizes, all_color_inds],axis=1))